In [ ]:
import numpy as np
import torch
from sklearn.model_selection import KFold
from torch import nn
from torch.utils.data import Dataset, DataLoader, Subset
from scipy.io import loadmat
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score



In [ ]:
class EEGDataset(Dataset):
    def __init__(self, data_path):
        mat_data = loadmat(data_path)
        data = mat_data['data']
        self.data = data[:, 0:272]
        self.labels = data[:, 272:273]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        return torch.from_numpy(sample).float(), torch.tensor(label).long()  

In [ ]:
dataset = EEGDataset(data_path="./your_dataset")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### LR

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline


feature_mapping = {}
with open('index.txt', 'r') as file:
    for line in file:
        parts = line.strip().split(' ')
        index = int(parts[0]) - 1  
        name = ' '.join(parts[1:])
        feature_mapping[index] = name


np.random.seed(0)


kf = KFold(n_splits=10, shuffle=True)


log_reg_model = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000, multi_class='multinomial'))


accuracies, recalls, precisions, f1s, balanced_accs, unweighted_f1s = [], [], [], [], [], []


all_coefs = []

for train_index, test_index in kf.split(dataset):
    
    train_dataset = torch.utils.data.Subset(dataset, train_index)
    test_dataset = torch.utils.data.Subset(dataset, test_index)

    
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

    
    X_train, y_train = [], []
    for data, label in train_loader:
        X_train.append(data.numpy())
        y_train.append(label.numpy())
    X_train = np.concatenate(X_train, axis=0)
    y_train = np.concatenate(y_train, axis=0).ravel()  

    
    log_reg_model.fit(X_train, y_train)
    

    
    X_test, y_test = [], []
    for data, label in test_loader:
        X_test.append(data.numpy())
        y_test.append(label.numpy())
    X_test = np.concatenate(X_test, axis=0)
    y_test = np.concatenate(y_test, axis=0).ravel()

    
    y_pred = log_reg_model.predict(X_test)

    
    accuracies.append(accuracy_score(y_test, y_pred))
    recalls.append(recall_score(y_test, y_pred, average='macro'))
    precisions.append(precision_score(y_test, y_pred, average='macro'))
    f1s.append(f1_score(y_test, y_pred, average='macro'))
    balanced_accs.append(balanced_accuracy_score(y_test, y_pred))
    unweighted_f1s.append(f1_score(y_test, y_pred, average='weighted'))

    
    all_coefs.append(log_reg_model.named_steps['logisticregression'].coef_)



metrics1 = {
    'Accuracy': (np.mean(accuracies), np.std(accuracies)),
    'Recall': (np.mean(recalls), np.std(recalls)),
    'Precision': (np.mean(precisions), np.std(precisions)),
    'F1 Score': (np.mean(f1s), np.std(f1s)),
    'Balanced Accuracy': (np.mean(balanced_accs), np.std(balanced_accs)),
    'Unweighted F1 Score': (np.mean(unweighted_f1s), np.std(unweighted_f1s))
}


for metric, values in metrics1.items():
    print(f"{metric} - Mean: {values[0]}, Standard Deviation: {values[1]}")


avg_coefs = np.mean(np.array(all_coefs), axis=0)


feature_importances = [(feature_mapping[i], coef) for i, coef in enumerate(avg_coefs[0])]
feature_importances.sort(key=lambda x: abs(x[1]), reverse=True)


feature_ranking = ["{}. feature {} (importance: {})".format(i+1, feat, imp) for i, (feat, imp) in enumerate(feature_importances)]

for feat in feature_ranking:
    print(feat)





### SVM

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, balanced_accuracy_score# 十折交叉验证 SVM

kf = KFold(n_splits=10, shuffle=True)

svm_model = SVC(kernel='rbf', C=1.0, gamma='scale')


np.random.seed(0)


accuracies, recalls, precisions, f1s, balanced_accs, unweighted_f1s = [], [], [], [], [], []

for train_index, test_index in kf.split(dataset):
    train_dataset = torch.utils.data.Subset(dataset, train_index)
    test_dataset = torch.utils.data.Subset(dataset, test_index)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

    X_train, y_train = [], []
    for data, label in train_loader:
        X_train.append(data.numpy())
        y_train.append(label.numpy())
    X_train = np.concatenate(X_train, axis=0)
    y_train = np.concatenate(y_train, axis=0).ravel()  

    svm_model.fit(X_train, y_train)

    X_test, y_test = [], []
    for data, label in test_loader:
        X_test.append(data.numpy())
        y_test.append(label.numpy())
    X_test = np.concatenate(X_test, axis=0)
    y_test = np.concatenate(y_test, axis=0).ravel()

    y_pred = svm_model.predict(X_test)

    accuracies.append(accuracy_score(y_test, y_pred))
    recalls.append(recall_score(y_test, y_pred, average='macro'))
    precisions.append(precision_score(y_test, y_pred, average='macro'))
    f1s.append(f1_score(y_test, y_pred, average='macro'))
    balanced_accs.append(balanced_accuracy_score(y_test, y_pred))
    unweighted_f1s.append(f1_score(y_test, y_pred, average='weighted'))

metrics2 = {
    'Accuracy': (np.mean(accuracies), np.std(accuracies)),
    'Recall': (np.mean(recalls), np.std(recalls)),
    'Precision': (np.mean(precisions), np.std(precisions)),
    'F1 Score': (np.mean(f1s), np.std(f1s)),
    'Balanced Accuracy': (np.mean(balanced_accs), np.std(balanced_accs)),
    'Unweighted F1 Score': (np.mean(unweighted_f1s), np.std(unweighted_f1s))
}

for metric, values in metrics2.items():
    print(f"{metric} - Mean: {values[0]}, Standard Deviation: {values[1]}")





### DT

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, balanced_accuracy_score

np.random.seed(0)


kf = KFold(n_splits=10, shuffle=True)

dt_model = DecisionTreeClassifier(min_samples_leaf=4)

accuracies, recalls, precisions, f1s, balanced_accs, unweighted_f1s = [], [], [], [], [], []

for train_index, test_index in kf.split(dataset):

    train_dataset = torch.utils.data.Subset(dataset, train_index)
    test_dataset = torch.utils.data.Subset(dataset, test_index)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)


    X_train, y_train = [], []
    for data, label in train_loader:
        X_train.append(data.numpy())
        y_train.append(label.numpy())
    X_train = np.concatenate(X_train, axis=0)
    y_train = np.concatenate(y_train, axis=0).ravel()  


    dt_model.fit(X_train, y_train)


    X_test, y_test = [], []
    for data, label in test_loader:
        X_test.append(data.numpy())
        y_test.append(label.numpy())
    X_test = np.concatenate(X_test, axis=0)
    y_test = np.concatenate(y_test, axis=0).ravel()


    y_pred = dt_model.predict(X_test)


    accuracies.append(accuracy_score(y_test, y_pred))
    recalls.append(recall_score(y_test, y_pred, average='macro'))
    precisions.append(precision_score(y_test, y_pred, average='macro'))
    f1s.append(f1_score(y_test, y_pred, average='macro'))
    balanced_accs.append(balanced_accuracy_score(y_test, y_pred))
    unweighted_f1s.append(f1_score(y_test, y_pred, average='weighted'))


metrics4 = {
    'Accuracy': (np.mean(accuracies), np.std(accuracies)),
    'Recall': (np.mean(recalls), np.std(recalls)),
    'Precision': (np.mean(precisions), np.std(precisions)),
    'F1 Score': (np.mean(f1s), np.std(f1s)),
    'Balanced Accuracy': (np.mean(balanced_accs), np.std(balanced_accs)),
    'Unweighted F1 Score': (np.mean(unweighted_f1s), np.std(unweighted_f1s))
}


for metric, values in metrics4.items():
    print(f"{metric} - Mean: {values[0]}, Standard Deviation: {values[1]}")


### RF

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, balanced_accuracy_score


feature_mapping = {}
with open('index.txt', 'r') as file:
    for line in file:
        parts = line.strip().split(' ')
        index = int(parts[0]) - 1  
        name = ' '.join(parts[1:])
        feature_mapping[index] = name


np.random.seed(0)


kf = KFold(n_splits=10, shuffle=True)


rf_model = RandomForestClassifier(
    max_features=0.3,
    bootstrap=True,
    max_samples=0.8,
    min_samples_leaf=3
)



accuracies, recalls, precisions, f1s, balanced_accs, unweighted_f1s = [], [], [], [], [], []

for train_index, test_index in kf.split(dataset):
    train_dataset = torch.utils.data.Subset(dataset, train_index)
    test_dataset = torch.utils.data.Subset(dataset, test_index)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)


    X_train, y_train = [], []
    for data, label in train_loader:
        X_train.append(data.numpy())
        y_train.append(label.numpy())
    X_train = np.concatenate(X_train, axis=0)
    y_train = np.concatenate(y_train, axis=0).ravel()  

    rf_model.fit(X_train, y_train)

    feature_importances = rf_model.feature_importances_
    indices = np.argsort(feature_importances)[::-1] 

    
    X_test, y_test = [], []
    for data, label in test_loader:
        X_test.append(data.numpy())
        y_test.append(label.numpy())
    X_test = np.concatenate(X_test, axis=0)
    y_test = np.concatenate(y_test, axis=0).ravel()

    
    y_pred = rf_model.predict(X_test)

    
    accuracies.append(accuracy_score(y_test, y_pred))
    recalls.append(recall_score(y_test, y_pred, average='macro'))
    precisions.append(precision_score(y_test, y_pred, average='macro'))
    f1s.append(f1_score(y_test, y_pred, average='macro'))
    balanced_accs.append(balanced_accuracy_score(y_test, y_pred))
    unweighted_f1s.append(f1_score(y_test, y_pred, average='weighted'))


metrics6 = {
    'Accuracy': (np.mean(accuracies), np.std(accuracies)),
    'Recall': (np.mean(recalls), np.std(recalls)),
    'Precision': (np.mean(precisions), np.std(precisions)),
    'F1 Score': (np.mean(f1s), np.std(f1s)),
    'Balanced Accuracy': (np.mean(balanced_accs), np.std(balanced_accs)),
    'Unweighted F1 Score': (np.mean(unweighted_f1s), np.std(unweighted_f1s))
}


for metric, values in metrics6.items():
    print(f"{metric} - Mean: {values[0]}, Standard Deviation: {values[1]}")


print("Feature ranking:")

for f in range(X_train.shape[1]):
    
    feature_name = feature_mapping[indices[f]]
    print(f"{f + 1}. feature {feature_name} (importance: {feature_importances[indices[f]]})")